<a href="https://colab.research.google.com/github/markbojic/parallel-alg/blob/main/Projekat2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 471kB 13.8MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4535546 sha256=4b33ef47b3196a7a8ebd11c3999a3ca18939ec82bc589b6010536fd301b91d3e
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=b1697ed96e5f1b64003e1ff4d8e4ab8fc29cdc78f6a479c2c1a5d5591ffe59a5
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=c141c86f098abb09dce8f8063e9d3df7118e4c0444c6e5672fb536755bc6e8c8
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
!nvidia-smi

Thu Jan  9 10:08:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import numpy as np
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit
import gc

kernel_code_template = """
__global__ void MatrixMulKernel(int *A, int *B, int m, int n, int colsA,  int *C)
{
    int row = threadIdx.x;
    int col = threadIdx.y;

    int sum = 0;

    if ((row < m) && (col < n)){
      for (int i = 0; i < colsA; ++i)
        sum += A[colsA*row + i] * B[i*n+col];
      C[row*n+col] = sum;
    }

}
"""

# Velicina matrica za mnozenje - Matrice dele MATRIX_SIZE_2
MATRIX_SIZE_1 = 5
MATRIX_SIZE_2 = 3
MATRIX_SIZE_3 = 4

# Popunjavanje matrica prosledjene velicine sa random elementima
a_cpu = np.random.randn(MATRIX_SIZE_1, MATRIX_SIZE_2).astype(np.int32)
b_cpu = np.random.randn(MATRIX_SIZE_2, MATRIX_SIZE_3).astype(np.int32)

# Prebacivanje matrica na gpu 
a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)

# Prazna matrica finalnih dimenizja - Koristi se za cuvanje rezultata mnozenja
c_gpu = gpuarray.empty((MATRIX_SIZE_1, MATRIX_SIZE_3), np.int32)

# Kompajlovanje kernel koda u SOurceMOdule
mod = compiler.SourceModule(kernel_code_template)

# Uzmi kernel funkciju uz mod-a
matrixmul = mod.get_function("MatrixMulKernel")

# Konvertovanje dimenzija kako bi se poslalo u kernel funkciju posle
row1 = np.int32(MATRIX_SIZE_1)
col1 = np.int32(MATRIX_SIZE_2)
row2 = np.int32(MATRIX_SIZE_2)
col2 = np.int32(MATRIX_SIZE_3)

# pozovi kernel
matrixmul(
    # In
    a_gpu, b_gpu, row1, col2, row2, 
    # Out
    c_gpu, 
    #koliko thredova da koristi blok
    block = (MATRIX_SIZE_1, MATRIX_SIZE_3, 1),
    )

# Stampanje rezultata mnozenja
print ("Matrica A:")
print (a_gpu)

print( "\n")
print("Matrca B:")
print(b_gpu)

print( "\n")
print( "Matrica C:")
print(c_gpu)

print( "\n")
print( "Ocekivana matrica:")
c_cpu = np.matmul(a_cpu, b_cpu)
print(c_cpu)

a_gpu.gpudata.free()
b_gpu.gpudata.free()
c_gpu.gpudata.free()
del a_gpu
del b_gpu
del c_gpu
del a_cpu
del b_cpu

#Zato sto zeza nakon nekoliko pustanja- javlja Illegal memory access (Logic Error), zato je gore i brisanje elemenata
#Force garbage Collector za svaki slucaj
gc.collect()

Matrica A:
[[-1  0  1]
 [ 0  0  1]
 [ 0  1  0]
 [-2 -1  0]
 [ 0 -1  0]]


Matrca B:
[[ 0  0  1  0]
 [-1 -1  0  0]
 [ 0  1  2  1]]


Matrica C:
[[ 0  1  1  1]
 [ 0  1  2  1]
 [-1 -1  0  0]
 [ 1  1 -2  0]
 [ 1  1  0  0]]


Ocekivana matrica:
[[ 0  1  1  1]
 [ 0  1  2  1]
 [-1 -1  0  0]
 [ 1  1 -2  0]
 [ 1  1  0  0]]


3163